In [ ]:
import os 
from automaticTB.tools import read_yaml, write_yaml
from automaticTB.solve.interaction import (
    get_InteractingAOSubspaces_from_cluster, CombinedAOSubspaceInteraction
)
from automaticTB.examples import get_cif_filename_for
from automaticTB.solve import (
    get_structure_from_cif_file,
)

bulksi = get_structure_from_cif_file(
    get_cif_filename_for("Si"), {"Si":"3s3p4s"}, rcut = 3.9
)
temporary_folder = "temporary"
all_iao = []
for i, centered_cluster in enumerate(bulksi.centered_clusters):
    temporary_file = os.path.join(temporary_folder, f"2nn_tmp_{i+1}.yaml")
    if os.path.exists(temporary_file):
        tmp_iao = read_yaml(temporary_file)
    else:
        tmp_iao = []
        for ceq_cluster in centered_cluster.centered_equivalent_clusters:
            ceq_cluster.set_symmetry(find_additional_symmetry = True)
            tmp_iao += get_InteractingAOSubspaces_from_cluster(ceq_cluster)
        write_yaml(tmp_iao, temporary_file)
    all_iao += tmp_iao
    
combined = CombinedAOSubspaceInteraction(all_iao)

free_aos = combined.free_AOpairs
for i, free in enumerate(free_aos):
    print(f"{i+1:>2d} " + str(free))

In [ ]:
# prepare bandstructure plot
from automaticTB.properties import UnitCell

cell = UnitCell(bulksi.cell)
path = cell.get_bandline(
    ["L   0.5   0.5  0.5  G  0.0   0.0  0.0",
     "G   0.0   0.0  0.0  X  0.5   0.0  0.5",
     "X   0.5   0.0  0.5  U  0.625 0.25 0.625",
     "K 0.375 0.375 0.75  G  0.0   0.0  0.0"]
)

In [ ]:
import numpy as np
from automaticTB.properties import (
    get_tbModel_from_structure_interactions_overlaps, BandStructureResult
)

values = np.array([
    -4.81341,    # 1  > Pair: Si-00      3s -> Si-00      3s r = (  0.00,  0.00,  0.00)
    0.00579/4,    # 2  > Pair: Si-00     3px -> Si-00      4s r = (  2.72,  2.72,  0.00)
    0.00000,    # 3  > Pair: Si-00      4s -> Si-00      4s r = (  0.00, -2.72,  2.72)
    1.77563,    # 4  > Pair: Si-00     3px -> Si-00     3px r = (  0.00,  0.00,  0.00)
    5.61342,    # 5  > Pair: Si-00      4s -> Si-00      4s r = (  0.00,  0.00,  0.00)
    -8.33255/4,    # 6  > Pair: Si-00      3s -> Si-01      3s r = (  1.36, -1.36,  1.36)
    0.08002/4,    # 7  > Pair: Si-00      3s -> Si-00     3px r = (  2.72,  2.72,  0.00)
    5.86140/4,    # 8  > Pair: Si-00      3s -> Si-01     3px r = (  1.36, -1.36,  1.36)
    0.00000,    # 9  > Pair: Si-00      3s -> Si-01      4s r = (  1.36, -1.36,  1.36)
    0.00000,    #10  > Pair: Si-00      3s -> Si-00      4s r = (  2.72,  2.72,  0.00)
    -5.86140/4,    #11  > Pair: Si-00     3px -> Si-01      3s r = (  1.36, -1.36,  1.36)
    5.29091/4,    #12  > Pair: Si-00     3px -> Si-01     3pz r = (  1.36, -1.36,  1.36)
    1.69916/4,    #13  > Pair: Si-00     3px -> Si-01     3px r = (  1.36, -1.36,  1.36)
    0.55067/4,    #14  > Pair: Si-00     3pz -> Si-00     3px r = (  2.72,  0.00,  2.72)
    0.00762/4,    #15  > Pair: Si-00     3px -> Si-00     3px r = ( -2.72,  2.72,  0.00)
    -4.88308/4,    #16  > Pair: Si-00     3px -> Si-01      4s r = (  1.36, -1.36,  1.36)
    -0.10662/4,    #17  > Pair: Si-00     3px -> Si-00     3px r = (  0.00, -2.72,  2.72)
    0.00000,    #18  > Pair: Si-00      4s -> Si-01      3s r = (  1.36, -1.36,  1.36)
    4.88308/4,    #19  > Pair: Si-00      4s -> Si-01     3px r = (  1.36, -1.36,  1.36)
    0.00000,    #20  > Pair: Si-00      4s -> Si-01      4s r = (  1.36, -1.36,  1.36)
    0.01591/4,    #21  > Pair: Si-00      3s -> Si-00      3s r = (  0.00, -2.72,  2.72)
])


interaction_pairs = combined.solve_interactions_to_InteractionPairs(values)
model = get_tbModel_from_structure_interactions_overlaps(bulksi, interaction_pairs)

# reference for DFT calculated band structure: 
# http://lampx.tugraz.at/~hadley/ss1/semiconductors/silicon_bandstructure.php
bandresult = BandStructureResult.from_tightbinding_and_kpath(model, path)
tmp_filename = "Si_2NN.pdf"
bandresult.plot_data(tmp_filename)
